Going to train some NNs on the different delta data I guess.... not sure how I'm gonna split it up yet but lets get started

In [1]:
#imports
import uproot as ur
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import vector as v

data_path = '/fast_scratch/atlas_images/v01-45/'

import atlas_mpl_style as ampl #makes matplotlib follow atlas formal style
ampl.use_atlas_style()

v.register_awkward()

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2" #specify GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve

import graph_util as gu
import plot_util as pu
import matplotlib.pyplot as plt

2021-07-14 12:02:47.497571: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
branches = ['cluster_hitsTruthIndex', 'cluster_hitsTruthE', 'truthPartPdgId', 'truthPartPt','truthPartEta', 
            'truthPartPhi', 'truthPartPt', 'truthPartMass']

events = ur.open(data_path+'delta_medium.root')

tree = events['EventTree']

In [3]:
the_branches = tree.arrays(expressions=branches)

To do this, I need to first set up the different selections, then do the data processing, (I guess the other way around works too? Okay I'm gonna change the convert_sets_delta file to also return truthID, then can do the selections on the pre-made data, then can run the models

In [4]:
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
eventNumber          | int64_t                  | AsDtype('>i8')
lumiBlock            | int32_t                  | AsDtype('>i4')
coreFlags            | uint32_t                 | AsDtype('>u4')
mcEventNumber        | int32_t                  | AsDtype('>i4')
mcChannelNumber      | int32_t                  | AsDtype('>i4')
mcEventWeight        | float                    | AsDtype('>f4')
nTruthPart           | int32_t                  | AsDtype('>i4')
G4PreCalo_n_EM       | int32_t                  | AsDtype('>i4')
G4PreCalo_E_EM       | float                    | AsDtype('>f4')
G4PreCalo_n_Had      | int32_t                  | AsDtype('>i4')
G4PreCalo_E_Had      | float                    | AsDtype('>f4')
truthVertexX         | float                    | AsDtype(

In [5]:
import convert_sets_delta #okay so this doesn't line up, why?

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data


IndexError: boolean index did not match indexed array along dimension 0; dimension is 751874 but corresponding boolean dimension is 740871

In [7]:
clus_E = tree.arrays('cluster_E')
truth_ID = tree.arrays('truthPartPdgId')

In [9]:
len(clus_E)

200000

In [10]:
len(truth_ID)

200000

In [11]:
clus_e = gu.loadVectorBranchFlat('cluster_E', tree)

In [12]:
len(clus_e)

740871

In [14]:
truid = gu.loadVectorBranchFlat('truthPartPdgId', tree)

In [15]:
len(truid)

751874

In [18]:
#okay so its because they're being flattened and are different lengths... 
print(truid)

[2114 2112  111 ... 2224 2212  211]


In [19]:
print(clus_e)

[2.0940566  0.79894364 0.5638675  ... 0.63663745 0.3209459  1.641062  ]


In [21]:
truthe = tree.arrays('truthPartE')

In [22]:
len(truthe)

200000

In [23]:
trueflat = gu.loadVectorBranchFlat('truthPartE', tree)

In [24]:
len(trueflat)

751874

In [25]:
clusID = gu.loadVectorBranchFlat('cluster_cell_ID', tree)

ValueError: in ListOffsetArray64, cannot convert to RegularArray because subarray lengths are not regular

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)

In [ ]:
clusID

#### okay I think I see how to do it: make separate datasets for each???? so load it all up here with all the different branches I'll need, do the selection from the last notebook to separate all the types of deltas and apply it to each different branch that I need, then save those in separate files to call the convert sets on. Could even save them in the same file and have like a flag/indicator as to which delta each event is, so that I could run convert_sets on multiple different types of delta at once? okay yay

In [26]:
geotree = events['CellGeo']

In [27]:
geotree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
cell_geo_ID          | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), he...
cell_geo_sampling    | std::vector<uint16_t>    | AsJagged(AsDtype('>u2'), he...
cell_geo_eta         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_phi         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_rPerp       | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_deta        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_dphi        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_volume      | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_sigma       | std::vector<float>       | AsJagged(AsDtype('>f4'), he...


In [28]:
cellgeoeta = geotree.arrays('cell_geo_eta')

In [30]:
len(cellgeoeta)

10

Okay so that confirms I don't have to mess with this stuff; recall this is all just its own thing, and it applies to any event, and is not event-specific, it just remains true for all events, it just describes the detector. okay.

In [32]:
rb = ['cluster_cell_ID', 'cluster_cell_E', 'cluster_Phi', 'cluster_Eta', 'cluster_Pt', 'truthPartPdgId',
               'cluster_E', 'cluster_ENG_CALIB_TOT'] #okay these are actually the only ones I need

In [43]:
brnsh = tree.arrays(expressions=rb)

In [33]:
#load up all the data we need
clus_cell_id = tree.arrays('cluster_cell_ID')
clus_cell_E = tree.arrays(rb[1])
clus_phi = tree.arrays(rb[2])
clus_eta = tree.arrays(rb[3])
clus_pt = tree.arrays(rb[4])
tru_id = tree.arrays(rb[5])
clus_E = tree.arrays(rb[6])
#I dont think I need calib?

In [40]:
tru_id = tree.arrays(expressions=rb[5])

In [46]:
#load up all data we need
tru_id = brnsh.truthPartPdgId
clus_cell_id = brnsh.cluster_cell_ID
clus_cell_E = brnsh.cluster_cell_E
clus_phi = brnsh.cluster_Phi
clus_eta = brnsh.cluster_Eta
clus_pt = brnsh.cluster_Pt
clus_E = brnsh.cluster_E

In [47]:
len(tru_id)

200000

In [48]:
np.array(tru_id[0])

array([2114, 2112,  111,   22,   22], dtype=int32)

In [49]:
#sort into particle types
ID1114 = []
cci1114 = []
ccE1114 = []
cph1114 = []
cet1114 = []
cpt1114 = []
cE1114 = []

IDn2214 = []
ccin2214 = []
ccEn2214 = []
cphn2214 = []
cetn2214 = []
cptn2214 = []
cEn2214 = []


IDn2114 = []
ccin2114 = []
ccEn2114 = []
cphn2114 = []
cetn2114 = []
cptn2114 = []
cEn2114 = []


IDn2224 = []
ccin2224 = []
ccEn2224 = []
cphn2224 = []
cetn2224 = []
cptn2224 = []
cEn2224 = []

ID2214 = []
cci2214 = []
ccE2214 = []
cph2214 = []
cet2214 = []
cpt2214 = []
cE2214 = []


ID2224 = []
cci2224 = []
ccE2224 = []
cph2224 = []
cet2224 = []
cpt2224 = []
cE2224 = []


IDn1114 = []
ccin1114 = []
ccEn1114 = []
cphn1114 = []
cetn1114 = []
cptn1114 = []
cEn1114 = []


ID2114 = []
cci2114 = []
ccE2114 = []
cph2114 = []
cet2114 = []
cpt2114 = []
cE2114 = []


for i in range(len(tru_id)):
    
    if (-2114 in tru_id[i]): 
        IDn2114.append(tru_id[i])
        ccin2114.append(clus_cell_id[i])
        ccEn2114.append(clus_cell_E[i])
        cphn2114.append(clus_phi[i])
        cetn2114.append(clus_eta[i])
        cptn2114.append(clus_pt[i])
        cEn2114.append(clus_E[i])

    elif (-2224 in tru_id[i]):
        IDn2224.append(tru_id[i])
        ccin2224.append(clus_cell_id[i])
        ccEn2224.append(clus_cell_E[i])
        cphn2224.append(clus_phi[i])
        cetn2224.append(clus_eta[i])
        cptn2224.append(clus_pt[i])
        cEn2224.append(clus_E[i])
        
    elif (2214 in tru_id[i]): 
        ID2214.append(tru_id[i])
        cci2214.append(clus_cell_id[i])
        ccE2214.append(clus_cell_E[i])
        cph2214.append(clus_phi[i])
        cet2214.append(clus_eta[i])
        cpt2214.append(clus_pt[i])
        cE2214.append(clus_E[i])
        
    elif (2224 in tru_id[i]):
        ID2224.append(tru_id[i])
        cci2224.append(clus_cell_id[i])
        ccE2224.append(clus_cell_E[i])
        cph2224.append(clus_phi[i])
        cet2224.append(clus_eta[i])
        cpt2224.append(clus_pt[i])
        cE2224.append(clus_E[i])
        
    elif (-1114 in tru_id[i]): 
        IDn1114.append(tru_id[i])
        ccin1114.append(clus_cell_id[i])
        ccEn1114.append(clus_cell_E[i])
        cphn1114.append(clus_phi[i])
        cetn1114.append(clus_eta[i])
        cptn1114.append(clus_pt[i])
        cEn1114.append(clus_E[i])
        
    elif (2114 in tru_id[i]):
        ID2114.append(tru_id[i])
        cci2114.append(clus_cell_id[i])
        ccE2114.append(clus_cell_E[i])
        cph2114.append(clus_phi[i])
        cet2114.append(clus_eta[i])
        cpt2114.append(clus_pt[i])
        cE2114.append(clus_E[i])
        
    elif (1114 in tru_id[i]): 
        ID1114.append(tru_id[i])
        cci1114.append(clus_cell_id[i])
        ccE1114.append(clus_cell_E[i])
        cph1114.append(clus_phi[i])
        cet1114.append(clus_eta[i])
        cpt1114.append(clus_pt[i])
        cE1114.append(clus_E[i])
        
    elif (-2214 in tru_id[i]):
        IDn2214.append(tru_id[i])
        ccin2214.append(clus_cell_id[i])
        ccEn2214.append(clus_cell_E[i])
        cphn2214.append(clus_phi[i])
        cetn2214.append(clus_eta[i])
        cptn2214.append(clus_pt[i])
        cEn2214.append(clus_E[i])

In [50]:
# I don't think I can just make a root file..... so this goes into the delta conv_sets I guess